# Index Return data

- using yahoo finance for historical stock return data 
- going to need the following
    - Dates of the announcement
    - Index returns 10 days before the announcement
    - Index returns the day of the announcement
    - Index returns 10 days after the announcement 

In [20]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


In [24]:
fomc_statements = pd.read_csv('dates/fomc_statements.csv')
fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
    'EURUSD=X',  # Euro/U.S. Dollar exchange rate
    'CHFUSD=X',  # Swiss Franc/U.S. Dollar exchange rate
    'JPYUSD=X'   # Japanese Yen/U.S. Dollar exchange rate
]

start_date = fomc_statements['Date'].min() - pd.Timedelta(days=15)
end_date = fomc_statements['Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()  
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for date in fomc_statements['Date']:
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker}
        for t in range(-10, 11):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

final_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker'] + [f'T{t:+}' for t in range(-10, 11)]
final_df = final_df[column_order]

final_df

# final_df.to_csv('long_format_output.csv', index=False)

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_90902/186129462.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********************

,announcement_date,ticker,T-10,T-9,T-8,T-7,T-6,T-5,T-4,T-3,...,T+1,T+2,T+3,T+4,T+5,T+6,T+7,T+8,T+9,T+10
0,2000-02-02,^GSPC,<NA>,-0.027634,0.006065,-0.004213,-0.003938,-0.027457,<NA>,<NA>,...,0.011248,-0.000421,<NA>,<NA>,-0.000091,0.012273,-0.020815,0.003627,-0.020969,<NA>
1,2000-02-02,^IXIC,<NA>,-0.032894,0.017414,-0.023396,-0.007457,-0.037749,<NA>,<NA>,...,0.033633,0.007875,<NA>,<NA>,0.018291,0.024465,-0.014514,0.02805,-0.020104,<NA>
2,2000-02-02,^DJI,<NA>,-0.021645,0.001973,0.000281,-0.000451,-0.02622,<NA>,<NA>,...,0.000931,-0.004507,<NA>,<NA>,-0.005291,0.004751,-0.023585,-0.00519,-0.020521,<NA>
3,2000-02-02,^RUT,<NA>,-0.020583,-0.002601,-0.001055,-0.007715,-0.023984,<NA>,<NA>,...,0.023025,0.007457,<NA>,<NA>,0.013073,0.009579,-0.002772,0.011586,-0.009424,<NA>
4,2000-02-02,^W5000,<NA>,-0.026098,0.005497,-0.004102,-0.004174,-0.028225,<NA>,<NA>,...,0.014137,-0.000158,<NA>,<NA>,0.002874,0.012514,-0.018755,0.005698,-0.018061,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4321,2025-03-19,^IRX,<NA>,-0.002859,0.00239,0.000477,-0.002144,0.0,<NA>,<NA>,...,-0.001909,0.000717,<NA>,<NA>,-0.000717,0.0,0.001913,-0.000477,0.0,<NA>
4322,2025-03-19,^TNX,<NA>,-0.024091,0.017802,0.006996,-0.01019,0.007955,<NA>,<NA>,...,-0.005404,0.004489,<NA>,<NA>,0.018579,-0.005541,0.007198,0.007146,-0.026093,<NA>
4323,2025-03-19,EURUSD=X,<NA>,0.006734,-0.001951,0.006898,-0.002558,-0.002812,<NA>,<NA>,...,-0.002521,-0.005059,<NA>,<NA>,-0.001929,-0.002971,-0.001489,-0.004211,0.005487,<NA>
4324,2025-03-19,CHFUSD=X,<NA>,0.006816,-0.003,-0.003916,0.001894,-0.000963,<NA>,<NA>,...,0.000394,-0.005647,<NA>,<NA>,-0.001748,0.000261,0.000045,-0.001708,0.003131,<NA>


# Sentiment Analysis 